In [1]:
!pip install XlsxWriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 1.7 MB/s eta 0:00:00


In [ ]:
!pip install dict

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import dict

In [ ]:
path = "/content/sp_500_stocks.csv"

In [ ]:
stocks = pd.read_csv(path)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [ ]:
symbol='AAPL'
api_url = f'https://ALGORITHMIC.iex.cloud/v1/data/core/quote/{symbol}?token=sk_604e2c9a98c947cc8048889f61234d07'
response = requests.get(api_url)
data = requests.get(api_url).json()
data

[{'avgTotalVolume': 73817592,
  'calculationPrice': 'tops',
  'change': -0.12,
  'changePercent': -0.0007,
  'close': None,
  'closeSource': 'official',
  'closeTime': None,
  'companyName': 'Apple Inc',
  'currency': 'USD',
  'delayedPrice': None,
  'delayedPriceTime': None,
  'extendedChange': None,
  'extendedChangePercent': None,
  'extendedPrice': None,
  'extendedPriceTime': None,
  'high': None,
  'highSource': None,
  'highTime': None,
  'iexAskPrice': 171.5,
  'iexAskSize': 300,
  'iexBidPrice': 169,
  'iexBidSize': 192,
  'iexClose': 170.73,
  'iexCloseTime': 1711465585272,
  'iexLastUpdated': 1711465585272,
  'iexMarketPercent': 0.009238545078451664,
  'iexOpen': 170.03,
  'iexOpenTime': 1711459800546,
  'iexRealtimePrice': 170.73,
  'iexRealtimeSize': 2,
  'iexVolume': 150674,
  'lastTradeTime': 1711465585272,
  'latestPrice': 170.73,
  'latestSource': 'IEX real time price',
  'latestTime': '11:06:25 AM',
  'latestUpdate': 1711465585272,
  'latestVolume': None,
  'low': Non

In [ ]:
data[0]['latestPrice']
data[0]['marketCap']

2636392343130

In [ ]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [ ]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL',
                                                   data[0]['latestPrice'],
                                                   data[0]['marketCap'],
                                                   'N/A'],
                                                  index = my_columns),
                                        ignore_index = True)
final_dataframe

<ipython-input-78-704cf149451c>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,170.73,2636392343130,N/A


In [153]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token=sk_604e2c9a98c947cc8048889f61234d07'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol,
                                                   data[0]['latestPrice'],
                                                   data[0]['marketCap'],
                                                   'N/A'],
                                                  index = my_columns),
                                        ignore_index = True)

<ipython-input-153-1e36ba287b0f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-153-1e36ba287b0f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-153-1e36ba287b0f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-153-1e36ba287b0f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-153-1e36ba287b0f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

In [ ]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,145.605,42670314627,N/A
1,AAL,15.015,9831173592,N/A
2,AAP,84.260,5017770799,N/A
3,AAPL,170.710,2636083505510,N/A
4,ABBV,179.270,317423884642,N/A
...,...,...,...,...
500,YUM,135.300,38064798684,N/A
501,ZBH,128.220,26295873301,N/A
502,ZBRA,298.260,15318633600,N/A
503,ZION,41.200,6081079624,N/A


In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [162]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings [:i]:
    batch_api_call_url = f'https://api.iex.cloud/v1/stock/market/batch?symbols={symbol_string}&types=quote&token=sk_604e2c9a98c947cc8048889f61234d07'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
            if symbol in data:
                symbol
                latest_price = data[symbol]['quote']['latestPrice']
                market_cap = data[symbol]['quote']['marketCap']
                final_dataframe = final_dataframe.append(
                    pd.Series([
                        symbol,
                        latest_price,
                        market_cap,
                        'N/A'
                    ],
                    index=my_columns),
                    ignore_index=True
                )
            else:
                print(f"No data found for symbol: {symbol}")

final_dataframe

<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

No data found for symbol: ABC
No data found for symbol: ANTM


<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

No data found for symbol: DISCA
No data found for symbol: FBHS


<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

...

<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

No data found for symbol: NLOK
No data found for symbol: RE


<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-162-014683a7a270>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

No data found for symbol: VIAC
No data found for symbol: WLTW


,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,144.705,42406564871,N/A
1,AAL,14.995,9818078456,N/A
2,AAP,84.600,5038018153,N/A
3,AAPL,170.225,2628594193225,N/A
4,ABBV,179.150,317211407004,N/A
...,...,...,...,...
486,XLNX,194.920,48414620999,N/A
487,XOM,114.350,453722996505,N/A
488,XRAY,32.665,6773521411,N/A
489,XRX,17.740,2202999430,N/A


In [163]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:10000000


In [164]:
position_size = val/ len(final_dataframe.index)
for i in range(0, len(final_dataframe.index) ):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,144.705,42406564871,140
1,AAL,14.995,9818078456,1358
2,AAP,84.600,5038018153,240
3,AAPL,170.225,2628594193225,119
4,ABBV,179.150,317211407004,113
...,...,...,...,...
486,XLNX,194.920,48414620999,104
487,XOM,114.350,453722996505,178
488,XRAY,32.665,6773521411,623
489,XRX,17.740,2202999430,1148


In [157]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [158]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [159]:
writer.sheets['Recommended Trades'].write('A1','Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1','Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1','Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1','Number of Shares to Buy', integer_format)

0

In [160]:
column_formats = {
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [161]:
writer.save()

<ipython-input-161-16d93d5523a4>:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
